In [7]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import os
from datetime import datetime

import itertools
from selenium.webdriver.chrome.options import Options

import requests
import time
from stem import Signal
from stem.control import Controller


In [2]:
rootDir = os.getcwd()
rootDir

'/home/xena/Desktop/prateek/oneGo/onego_poc1/main/web_scraping'

### Create list of indian and foreign airports using data in file airports.py

In [3]:
airports_file = r'/home/xena/Desktop/prateek/oneGo/onego_poc1/files/airports.py'
airports_dict = {}
indian_airports = []
foreign_airports = []
with open(airports_file) as f:      # using this code takes care op closing the file
          for line in f:
            airport_key = line.split(' = ')[0]
            airport_details = line.split(' = ')[1]
            airports_dict[airport_key] = eval(airport_details)

for value in airports_dict.values():
    if (value.get('country')=='India'):
        indian_airport = "%s-%s"%(value.get('code'),value.get('city'))
        indian_airports.append(indian_airport)
    else:
        foreign_airport = "%s-%s"%(value.get('code'),value.get('city'))
        foreign_airports.append(foreign_airport)
        
print("indian_airports list size: %d" % len(indian_airports))
print("foreign_airports list size: %d" % len(foreign_airports))    
    
            
          



indian_airports list size: 133
foreign_airports list size: 9354


In [4]:
print(indian_airports)


['AGR-Agra', 'AGX-Agatti Island', 'AJL-Aizawl', 'AKD-Akola', 'AMD-Ahmedabad', 'ATQ-Amritsar', 'BBI-Bhubaneswar', 'BDQ-Vadodara', 'BEK-Bareli', 'BEP-Bellary', 'BHJ-Bhuj', 'BHO-Bhopal', 'BHU-Bhavnagar', 'BKB-Bikaner', 'BLR-Bangalore', 'BOM-Mumbai', 'BUP-Bhatinda', 'CBD-Car Nicobar', 'CCJ-Calicut', 'CCU-Kolkata', 'CDP-Cuddapah', 'CJB-Coimbatore', 'COH-Cooch Behar', 'COK-Kochi', 'DAE-Daparizo', 'DAI-Darjeeling', 'DBD-Dhanbad', 'DED-Dehra Dun', 'DEL-New Delhi', 'DEP-Deparizo', 'DHM-Dharamshala', 'DIB-Dibrugarh', 'DIU-Diu', 'DMU-Dimapur', 'GAU-Guwahati', 'GAY-Gaya', 'GOI-Goa', 'GOP-Gorakhpur', 'GUX-Guna', 'GWL-Gwalior', 'HBX-Hubli', 'HJR-Khajuraho', 'HSS-Hissar', 'HYD-Hyderabad', 'IDR-Indore', 'IMF-Imphal', 'ISK-Nasik', 'IXA-Agartala', 'IXB-Siliguri', 'IXC-Chandigarh', 'IXD-Allahabad', 'IXE-Mangalore', 'IXG-Belgaum', 'IXH-Kailashahar', 'IXI-Lilabari', 'IXJ-Jammu', 'IXK-Keshod', 'IXL-Leh', 'IXM-Madurai', 'IXN-Khowai', 'IXP-Pathankot', 'IXQ-Kamalpur', 'IXR-Ranchi', 'IXS-Silchar', 'IXT-Pasighat

In [15]:
# url = "https://paytm.com/flights/flightSearch/AGR-Agra/AGX-Agatti Island/1/0/0/E/2019-05-30"
# url = "https://paytm.com/flights/flightSearch/STV-Surat/TIR-Tirupati/1/0/0/E/2019-05-30"
url = "https://paytm.com/flights/flightSearch/BLR-Bengaluru/BOM-Mumbai/1/0/0/E/2019-09-01"
data = pd.DataFrame(columns=['RecordDate','Airline','Flight','Dept_Date','Dept_Time','Origin','Duration','Stops','Arr_Date','Arr_Time','Destination','Total_Fare'])
files_dir = '/home/xena/Desktop/prateek/oneGo/onego_poc1/files/scraped_data'
filename = 'paytm_flights.csv'
data.to_csv(files_dir+'/'+ filename, mode='a', index=False)
DepDate = "2019-09-01"

def renew_tor_ip():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="birth@1992")
        controller.signal(Signal.NEWNYM)   

def scrape_and_save(url):
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    #driver = webdriver.Firefox()
    driver = webdriver.Chrome('/home/xena/Desktop/prateek/oneGo/onego_poc1/files/chromedriver',options=chrome_options)
    driver.implicitly_wait(30)
    driver.get(url)

    div_elem = driver.find_elements_by_css_selector('._3215.row') # . has to be put
#     print("div_elem size:{}".format(len(div_elem)))
#     print("div_elem:{}".format(div_elem))

    if len(div_elem)!=0:
        RecordDate = str(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
        
        ArrDate ="NA"
        rows=[]
        for elem in div_elem:
            row = elem.text.split("\n")   # list of values
            row=row[:-3]              # discard last three elements 
            row.insert(0,RecordDate)
            row.insert(3,DepDate)
            row.insert(8,ArrDate)
            rows.append(row)
#         print(len(rows))

        data = pd.DataFrame(rows)
        print(data.shape)
        data.to_csv(files_dir+'/' + filename, mode='a', index=False, header = False)
        

### Create urls for each pair of airports and call scrape_and_save method

In [16]:
cnt = 0
for pair in itertools.permutations(indian_airports,2):
    if(cnt>0):
        break
        
    renew_tor_ip()
    origin = pair[0]
    destination = pair[1]
#     url = "https://paytm.com/flights/flightSearch/%s/%s/1/0/0/E/2019-05-30" %(origin,destination)
    scrape_and_save(url)
    cnt=cnt+1
    print("url:{}".format(url))
#     print("cnt:{}".format(cnt))

(131, 12)
url:https://paytm.com/flights/flightSearch/BLR-Bengaluru/BOM-Mumbai/1/0/0/E/2019-09-01


In [33]:
import requests
import time
from stem import Signal
from stem.control import Controller


def get_current_ip():
    session = requests.session()

    # TO Request URL with SOCKS over TOR
    session.proxies = {}
    session.proxies['http']='socks5h://localhost:9050'
    session.proxies['https']='socks5h://localhost:9050'

    try:
        r = session.get('http://httpbin.org/ip')
    except Exception as e:
        print(str(e))
    else:
        return r.text


def renew_tor_ip():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate(password="birth@1992")
        controller.signal(Signal.NEWNYM)


if __name__ == "__main__":
    for i in range(5):
        print(get_current_ip())
        renew_tor_ip()
        time.sleep(5)

{
  "origin": "37.187.129.166, 37.187.129.166"
}

{
  "origin": "192.42.116.25, 192.42.116.25"
}

{
  "origin": "185.220.102.7, 185.220.102.7"
}

{
  "origin": "185.220.102.7, 185.220.102.7"
}

{
  "origin": "199.249.230.114, 199.249.230.114"
}



In [35]:
from torrequest import TorRequest
tr=TorRequest(password='birth@1992')
# Let’s check our current IP address

import requests
response= requests.get('http://ipecho.net/plain')
print ("My Original IP Address:",response.text)

tr.reset_identity() #Reset Tor
response= tr.get('http://ipecho.net/plain')
print ("New Ip Address",response.text)

My Original IP Address: 171.76.120.214
New Ip Address 194.71.109.44


In [ ]:
My Original IP Address: 171.76.120.214
New Ip Address 199.249.230.70

In [3]:
# import requests
# from lxml.html import fromstring
# def get_proxies():
#     url = 'https://free-proxy-list.net/'
#     response = requests.get(url)
#     parser = fromstring(response.text)
#     proxies = set()
#     for i in parser.xpath('//tbody/tr')[:10]:
#         if i.xpath('.//td[7][contains(text(),"yes")]'):
#             #Grabbing IP and corresponding PORT
#             proxy = ":".join([i.xpath('.//td[1]/text()')[0], i.xpath('.//td[2]/text()')[0]])
#             proxies.add(proxy)
#     return proxies

In [4]:
# proxies = get_proxies()
# print(proxies)

SSLError: HTTPSConnectionPool(host='free-proxy-list.net', port=443): Max retries exceeded with url: / (Caused by SSLError(SSLError("bad handshake: SysCallError(10054, 'WSAECONNRESET')")))

In [12]:
# PROXY = "165.22.57.60:8888" # IP:PORT or HOST:PORT

# driver = webdriver.Chrome('C:/Self_Work/oneGo/onego_poc1/files/chromedriver',options=chrome_options))
# chrome_options = webdriver.ChromeOptions()
# chrome_options.add_argument('--proxy-server=%s' % PROXY)

# # chrome = webdriver.Chrome(options=chrome_options)
# chrome.get("http://whatismyipaddress.com")

SyntaxError: invalid syntax (<ipython-input-12-cffcff772155>, line 3)